# Introduction to Cellmage

This notebook introduces the basic functionality of the `cellmage` library, a toolkit for interacting with Large Language Models in Jupyter notebooks.

**Date:** April 24, 2025

## What is Cellmage?

Cellmage is a library that simplifies interactions with various LLMs while providing:
- Conversation history management
- Persona configuration
- Snippet management
- Conversation saving/loading

## Setup & Installation

Let's start by ensuring we have the cellmage package installed. In this notebook, we'll use the existing installation from the local development environment.

In [1]:
# Add the parent directory to sys.path so we can import cellmage
import os
import sys
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)

# Ensure the cellmage package can be imported
# Get the absolute path of the current working directory
notebook_dir = os.getcwd()
# Get the project root directory (parent of the notebook directory)
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))

print(f"Notebook directory: {notebook_dir}")
print(f"Project root directory: {project_root}")

if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added path: {project_root}")

try:
    # Import cellmage
    import cellmage

    # Check version - handle case where __version__ might not be available
    try:
        print(f"Cellmage version: {cellmage.__version__}")
    except AttributeError:
        print("Cellmage imported successfully, but version information is not available")
except ModuleNotFoundError as e:
    print(f"Error importing cellmage: {e}")
    print("\nDebug information:")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Python path: {sys.path}")
    print("\nTry running this notebook from the project root directory")

2025-04-24 06:21:05,758 - cellmage - INFO - Cellmage logging initialized
2025-04-24 06:21:05,767 - cellmage.config - INFO - python-dotenv not installed, skipping .env file loading


Notebook directory: /Users/tpinto/madpin/cellmage/notebooks
Project root directory: /Users/tpinto/madpin/cellmage
Added path: /Users/tpinto/madpin/cellmage
Cellmage version: 0.1.0


## Setting Up the LLM Client

Cellmage uses an adapter pattern to work with multiple LLM providers. The new `DirectLLMAdapter` provides direct HTTP communication with LLM APIs without requiring additional packages.

Let's set up our LLM client to use OpenAI's GPT models:

In [2]:
from cellmage.adapters.direct_client import DirectLLMAdapter

# Create an LLM client with GPT-4.1-nano model
llm_client = DirectLLMAdapter(default_model="gpt-4.1-nano")

# You'll need to ensure your API key is set up in your environment
# For OpenAI: export CELLMAGE_API_KEY=your_api_key
# For testing we'll simulate a successful initialization
print("LLM client initialized with model: gpt-4.1-nano")

2025-04-24 06:21:05,842 - cellmage.adapters.direct_client - INFO - [Override] Setting 'api_key' = sk-L...xxmA
2025-04-24 06:21:05,843 - cellmage.adapters.direct_client - INFO - [Override] Setting 'api_base' = https://litellm.oracle.madpin.dev
2025-04-24 06:21:05,843 - cellmage.adapters.direct_client - INFO - [Override] Setting 'model' = gpt-4.1-nano


LLM client initialized with model: gpt-4.1-nano


## Creating a Chat Manager

The `ChatManager` is the central class in Cellmage that coordinates between all components. Let's create a basic setup:

In [3]:
# Create components for the chat manager
from cellmage.resources.memory_loader import MemoryLoader
from cellmage.storage.memory_store import MemoryStore

# Create in-memory components for testing
persona_loader = MemoryLoader()
snippet_provider = MemoryLoader()
history_store = MemoryStore()

# Create a basic persona
persona_loader.add_persona(
    name="helpful_assistant",
    system_message="You are a helpful assistant who provides clear, concise answers.",
    config={"temperature": 0.7},
)

# Create a chat manager
chat_manager = cellmage.ChatManager(
    llm_client=llm_client,
    persona_loader=persona_loader,
    snippet_provider=snippet_provider,
    history_store=history_store,
)

# Set default persona
chat_manager.set_default_persona("helpful_assistant")

print("Chat manager initialized with 'helpful_assistant' persona")

2025-04-24 06:21:05,887 - cellmage.resources.memory_loader - INFO - Added persona 'helpful_assistant' to memory
2025-04-24 06:21:05,888 - cellmage.chat_manager - INFO - Initializing ChatManager
2025-04-24 06:21:05,888 - cellmage.config - INFO - python-dotenv not installed, skipping .env file loading
2025-04-24 06:21:05,889 - cellmage.chat_manager - INFO - ChatManager initialized
2025-04-24 06:21:05,889 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.7
2025-04-24 06:21:05,889 - cellmage.chat_manager - INFO - Default persona set to 'helpful_assistant'


Chat manager initialized with 'helpful_assistant' persona


## Sending Messages

Now that our chat manager is set up, we can send messages to the LLM:

In [4]:
# Send a message to the LLM
response = chat_manager.chat(
    "What are the main features of the cellmage library?",
    stream=True,  # Enable streaming responses
)

2025-04-24 06:21:05,894 - cellmage.chat_manager - INFO - Sending message to LLM with 2 messages in context
2025-04-24 06:21:05,895 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 2 messages


## Adding a Code Snippet

Cellmage allows you to add code snippets to the conversation, which can be useful for providing context:

In [5]:
# Create a code snippet
snippet_provider.add_snippet(
    name="python_example",
    content="""```python
def calculate_fibonacci(n):
    if n <= 0:
        return 0
    elif n == 1:
        return 1
    else:
        return calculate_fibonacci(n-1) + calculate_fibonacci(n-2)
```""",
)

# Add the snippet to the conversation
chat_manager.add_snippet("python_example")

# Now ask about the snippet
response = chat_manager.chat(
    "Can you explain this fibonacci function and suggest how to make it more efficient?", stream=True
)

2025-04-24 06:21:07,177 - cellmage.resources.memory_loader - INFO - Added snippet 'python_example' to memory
2025-04-24 06:21:07,204 - cellmage.chat_manager - INFO - Added snippet 'python_example' as system message
2025-04-24 06:21:07,218 - cellmage.chat_manager - INFO - Sending message to LLM with 5 messages in context
2025-04-24 06:21:07,229 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 5 messages


## Viewing Conversation History

You can view the conversation history at any time:

In [6]:
# Get the conversation history
history = chat_manager.get_history()

# Print the history in a readable format
for i, message in enumerate(history):
    print(f"{i + 1}. {message.role}: {message.content[:50]}...")
    print(f"   ID: {message.id}")
    print()

1. system: You are a helpful assistant who provides clear, co...
   ID: 4971705f-c997-4081-a75a-b92f038415d1

2. user: What are the main features of the cellmage library...
   ID: a15b0e1b-4bd7-41f5-b595-ccd2c8dd9aa9

3. assistant: ...
   ID: 7fd5ebf1-77ba-4916-b220-16cc7315435f

4. system: ```python
def calculate_fibonacci(n):
    if n <= ...
   ID: c2bd3e5d-839a-4c79-8e65-948705f9ffca

5. user: Can you explain this fibonacci function and sugges...
   ID: 5c6333ad-39f9-4ed0-ae83-dace731a6073

6. assistant: ...
   ID: aeffcc08-a9b8-4724-aba0-8e1acfc73941



## Saving and Loading Conversations

Cellmage allows you to save and load conversations:

In [7]:
# Save the conversation
save_path = chat_manager.save_conversation("example_conversation")
print(f"Conversation saved to: {save_path}")

# Clear the history
chat_manager.clear_history()
print(f"History cleared, current message count: {len(chat_manager.get_history())}")

# Load the conversation back
if save_path:
    chat_manager.load_conversation(save_path)
    print(f"Conversation loaded, message count: {len(chat_manager.get_history())}")

2025-04-24 06:21:13,815 - cellmage.storage.memory_store - INFO - Saved conversation to memory with ID: example_conversation
2025-04-24 06:21:13,821 - cellmage.history_manager - INFO - History cleared. Kept 2 system messages.
2025-04-24 06:21:13,825 - cellmage.history_manager - INFO - Loaded conversation from example_conversation with 6 messages


Conversation saved to: example_conversation
History cleared, current message count: 2
Conversation loaded, message count: 6


## Conclusion

This notebook has demonstrated the basic functionality of the Cellmage library:
- Setting up an LLM client
- Creating and using personas
- Adding code snippets
- Sending messages to the LLM
- Viewing conversation history
- Saving and loading conversations

In the next notebooks, we'll explore more advanced features and test with different LLM models.